## Skigebiete scrapen

In [3]:
# Libraries laden
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

Einzelne Seiten zu Skigebieten absaugen, Öffnungszeiten abspeichern, bzw. Fehler loggen

In [16]:
path = "/Users/leasenn/Google Drive/CAS Datenjournalismus/skigebiete/"
linkliste = []
df_skigebiete = pd.read_csv(path+"skigebiete_linkliste.csv") # Bereits kreierte Liste von Skigebieten verwenden
linkliste = df_skigebiete["skigebiete"].values.tolist()

skigebiete_final = []
errors = []
now = datetime.datetime.now()
zeitstempel = now.strftime("%Y-%m-%d_%H-%M")

for gebiet_link in linkliste[:1]:
    print(gebiet_link)
    try:
        website = requests.get("https://www.bergfex.ch"+gebiet_link).text
        soup = BeautifulSoup(website, "html.parser")
        
        # Titel des Skigebiets
        '''titel = soup.find('h1')
        titel = (titel.text.replace("Skigebiet ", ""))
        print(titel)'''
        # Offene Skilifte auslesen
        liveangaben = soup.find_all('table', class_="schneewerte")

        if len(liveangaben) > 0: # Wenn es überhaupt Live-Angaben hat zum entsprechenden Skigebiet...
            for skigebiet in liveangaben:
                offenelifte = (skigebiet.find_all('td')[-1].text).split(" von ") # Auslesen im Format "12 von 24" (offene Lifte sind immer das letzte TD, deshalb -1)
                total = offenelifte[1].strip()
                offen = offenelifte[0].strip()

                # Zeitquelle angeben und formatieren, für "Heute" und "Gestern" richtiges Datum einsetzen...
                aktualisiertam = skigebiet.find_all('td')[0].text.replace("Heute", now.strftime("%d.%m.%Y")).replace("Gestern", (now-datetime.timedelta(days=1)).strftime("%d.%m.%Y"))
                # bzw. für "Mo", "Di" und Co. Buchstaben rausnehmen und Jahreszahl anhängen
                tage = ["Mo", "Di", "Mi", "Do", "Fr", "Sa", "So"]
                for tag in tage:
                    if(tag in aktualisiertam):
                        teile_datum = aktualisiertam.replace(tag,"").split(", ")
                        aktualisiertam = teile_datum[1]+now.strftime("%Y")+", "+teile_datum[2]

                skigebiete_final.append({"zeit_scraping": zeitstempel, "zeit_siteaktualisiert": aktualisiertam, "titel": gebiet_link, "totallifte": total, "offen": offen})
    except:
        errors.append({"status": "failed", "gebiet": gebiet_link, "zeitstempel" : zeitstempel})


/sedrun/


In [14]:
print(skigebiete_final)

[]


Alles abspeichern

In [6]:
pd.DataFrame(skigebiete_final).to_csv(path+'skigebiete_oeffnungszeiten/oeffnungszeiten_'+zeitstempel+".csv")
pd.DataFrame(errors).to_csv(path+'skigebiete_oeffnungszeiten/errors_'+zeitstempel+".csv")